In [1]:
spark

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 6, Finished, Available)

In [15]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 20, Finished, Available)

## Import Libraries

In [ ]:
import os
from pyspark.sql.functions import col, count, when
import pandas as pd
from pyspark.sql.functions import length
from pyspark.sql import functions as F

## Read Data

In [16]:
datastore = 'azureml://datastores/workspaceblobstore/paths/'

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 21, Finished, Available)

In [17]:
submissions_path = 'filtered-submissions'
submissions_df = spark.read.parquet(f"{datastore}{submissions_path}")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 22, Finished, Available)

## Data Processing:

In [6]:
PLOT_DIR = os.path.join("Users/sk2224/fall-2023-reddit-project-team-34/data", "plots")
CSV_DIR = os.path.join("Users/sk2224/fall-2023-reddit-project-team-34/data", "csv")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 11, Finished, Available)

Drop unnecessary columns:

In [5]:
# # List of columns to be dropped
# columns_to_drop = [
#     'promoted', 'promoted_by', 'promoted_display_name', 'promoted_url',
#     'secure_media', 'secure_media_embed', 'thumbnail', 'thumbnail_height', 
#     'thumbnail_width', 'third_party_trackers', 'third_party_tracking', 
#     'third_party_tracking_2', 'source', 'retrieved_on', 'suggested_sort', 
#     'spoiler', 'stickied', 'whitelist_status'
# ]

# # Dropping the columns
# submissions_df = submissions_df.drop(*columns_to_drop)

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 10, Finished, Available)

Shape of the data:

In [4]:
# submissions_row_count = submissions_df.count()
# submissions_col_count = len(submissions_df.columns)
# print(f"Shape of the submissions dataframe is {submissions_row_count:,}x{submissions_col_count}")

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 9, Finished, Available)

Breakdown of subreddit counts:

In [62]:
# subreddit_count_df = submissions_df.groupBy('subreddit').count().toPandas()

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 67, Finished, Available)

In [6]:
submissions_df.groupBy('subreddit').count().show()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 35, 11, Finished, Available)

+--------------------+------+
|           subreddit| count|
+--------------------+------+
|               anime|404298|
|          television| 89586|
|televisionsuggest...|  7991|
|              movies|382085|
|        Animesuggest| 74101|
|    MovieSuggestions| 58907|
+--------------------+------+



In [65]:
# subreddit_count_df.to_csv(f"{CSV_DIR}/subreddit_count.csv")

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 70, Finished, Available)

Number of partitions in RDD:

In [6]:
# get number of partitions
submissions_df.rdd.getNumPartitions()

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 11, Finished, Available)

100

Columns in the data:

In [7]:
# print schema
submissions_df.printSchema()

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 12, Finished, Available)

root
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- brand_safe: boolean (nullable = true)
 |-- contest_mode: boolean (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- crosspost_parent: string (nullable = true)
 |-- crosspost_parent_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |-- approved_by: string (nullable = true)
 |    |    |-- archived: boolean (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |-- author_flair_text: string (nullable = true)
 |    |    

Subset the data for faster processing:

In [6]:
submissions_df_small = submissions_df.sample(withReplacement=False, fraction=0.01)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 40, 11, Finished, Available)

In [7]:
# take a subset of columns
df = submissions_df_small.select("subreddit", "author", "title", "selftext",
                             "created_utc", "num_comments", "score", 
                             "over_18", "media", "pinned", "locked", 
                             "disable_comments", "domain", "hidden", 
                             "distinguished", "hide_score")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 40, 12, Finished, Available)

In [18]:
# take a subset of columns
df = submissions_df.select("subreddit", "author", "title", "selftext",
                             "created_utc", "num_comments", "score", 
                             "over_18", "media", "pinned", "locked", 
                             "disable_comments", "domain", "hidden", 
                             "distinguished", "hide_score")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 23, Finished, Available)

Count the missing values in data:

In [12]:
# find missing values
missing_vals = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 17, Finished, Available)

In [13]:
missing_vals.show()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 18, Finished, Available)

+---------+------+-----+--------+-----------+------------+-----+-------+------+------+------+----------------+------+------+-------------+----------+
|subreddit|author|title|selftext|created_utc|num_comments|score|over_18| media|pinned|locked|disable_comments|domain|hidden|distinguished|hide_score|
+---------+------+-----+--------+-----------+------------+-----+-------+------+------+------+----------------+------+------+-------------+----------+
|        0|     0|    0|       0|          0|           0|    0|      0|892405|     0|     0|         1016968|  8952|     0|      1015530|         0|
+---------+------+-----+--------+-----------+------------+-----+-------+------+------+------+----------------+------+------+-------------+----------+



In [15]:
# Manually inputting data
data = {
    'subreddit': [0],
    'author': [0],
    'title': [0],
    'selftext': [0],
    'created_utc': [0],
    'num_comments': [0],
    'score': [0],
    'over_18': [0],
    'media': [892405],
    'pinned': [0],
    'locked': [0],
    'disable_comments': [1016968],
    'domain': [8952],
    'hidden': [0],
    'distinguished': [1015530],
    'hide_score': [0]
}

# Creating a Pandas DataFrame
missing_vals_pd = pd.DataFrame(data)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 20, Finished, Available)

In [16]:
missing_vals_pd

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 21, Finished, Available)

,subreddit,author,title,selftext,created_utc,num_comments,score,over_18,media,pinned,locked,disable_comments,domain,hidden,distinguished,hide_score
0,0,0,0,0,0,0,0,0,892405,0,0,1016968,8952,0,1015530,0


In [13]:
# missing_vals_pd = missing_vals.toPandas()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 35, 18, Finished, Available)

In [17]:
# Convert the DataFrame to long format
df_long = missing_vals_pd.melt(var_name='Column', value_name='Missing Values')

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 22, Finished, Available)

In [18]:
df_long

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 23, Finished, Available)

,Column,Missing Values
0,subreddit,0
1,author,0
2,title,0
3,selftext,0
4,created_utc,0
5,num_comments,0
6,score,0
7,over_18,0
8,media,892405
9,pinned,0


In [20]:
df_long.to_csv(f"{CSV_DIR}/num_missing_val.csv", index=False)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 25, Finished, Available)

### Feature Engineering

Counting the length of the posted submission:

In [19]:
# calculate post length
df = df.withColumn('post_length', length(df.title) + length(df.selftext))

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 24, Finished, Available)

Extracting meaningful information from datetime column:

In [20]:
# convert to datetime
df = df.withColumn('created_utc', F.to_timestamp('created_utc'))

# Extract time-based features
df = df.withColumn('hour_of_day', F.hour('created_utc'))
df = df.withColumn('day_of_week', F.dayofweek('created_utc'))  # 1 (Sunday) to 7 (Saturday)
# Map each day of the week from numeric to string
df = df.withColumn('day_of_week_str', F.expr("""
    CASE day_of_week 
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
    END
"""))
df = df.withColumn('day_of_month', F.dayofmonth('created_utc'))
df = df.withColumn('month', F.month('created_utc'))
df = df.withColumn('year', F.year('created_utc'))

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 25, Finished, Available)

Converting media column to boolean:

In [21]:
# convert missing values to False
df = df.withColumn('has_media', F.col('media').isNotNull())

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 26, Finished, Available)

In [22]:
# drop unnecessary columns
df = df.drop(*["media", "created_utc", "disable_comments", "distinguished"])

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 27, Finished, Available)

In [23]:
df.printSchema()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 28, Finished, Available)

root
 |-- subreddit: string (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- num_comments: long (nullable = true)
 |-- score: long (nullable = true)
 |-- over_18: boolean (nullable = true)
 |-- pinned: boolean (nullable = true)
 |-- locked: boolean (nullable = true)
 |-- domain: string (nullable = true)
 |-- hidden: boolean (nullable = true)
 |-- hide_score: boolean (nullable = true)
 |-- post_length: integer (nullable = true)
 |-- hour_of_day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- day_of_week_str: string (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- has_media: boolean (nullable = false)



## Read CSVs exported from the Spark Job

In [11]:
# Define the path to your CSV file
csv_file_path_1 = f"{datastore}csv/subreddit_count_eda.csv"
csv_file_path_2 = f"{datastore}csv/num_missing_val_eda.csv"
csv_file_path_3 = f"{datastore}csv/datetime_counts_eda.csv"

# Read the CSV file using PySpark
df_subreddit_count = spark.read.csv(csv_file_path_1, header=True, inferSchema=True)
df_num_missing = spark.read.csv(csv_file_path_2, header=True, inferSchema=True)
df_datetime = spark.read.csv(csv_file_path_3, header=True, inferSchema=True)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 16, Finished, Available)

In [12]:
df_subreddit_count_pd = df_subreddit_count.toPandas()
df_num_missing_pd = df_num_missing.toPandas()
df_datetime_pd = df_datetime.toPandas()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 17, Finished, Available)

# Data for Time analysis

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.4.0

# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       h06a4308_0         123 KB
    certifi-2023.11.17         |  py310h06a4308_0         158 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/

In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

3.4.0


In [10]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, length, expr, to_timestamp, hour, dayofweek, dayofmonth, month, year, avg
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F
from pyspark.sql.functions import desc
from pyspark.sql.functions import rank
from pyspark.sql.functions import length

### Import Data

In [5]:
%%time
bucket = "project-group34"
session = sagemaker.Session()
output_prefix_data_submissions = "project/submissions/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_submissions}"
print(f"reading comments from {s3_path}")
submissions = spark.read.parquet(s3_path, header=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://project-group34/project/submissions/yyyy=*


23/12/06 18:18:23 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 54.7 ms, sys: 8.5 ms, total: 63.2 ms
Wall time: 5.93 s


23/12/06 18:18:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
submissions = submissions.select("subreddit", "score", "created_utc")

In [7]:
submissions.show(5)

+----------+-----+-------------------+
| subreddit|score|        created_utc|
+----------+-----+-------------------+
|television|    0|2021-01-27 22:16:47|
|     anime|    0|2021-01-27 22:17:30|
|television|    4|2021-01-27 22:17:42|
|    movies|    0|2021-01-27 22:18:23|
|     anime|    0|2021-01-27 22:19:23|
+----------+-----+-------------------+
only showing top 5 rows



In [8]:
# Feature engineering
submissions = submissions.withColumn('created_utc', to_timestamp('created_utc'))
submissions = submissions.withColumn('hour_of_day', hour('created_utc'))
submissions = submissions.withColumn('day_of_week', dayofweek('created_utc'))
submissions = submissions.withColumn('day_of_week_str', expr("""
    CASE day_of_week 
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
    END
"""))
submissions = submissions.withColumn('day_of_month', dayofmonth('created_utc'))
submissions = submissions.withColumn('month', month('created_utc'))
submissions = submissions.withColumn('year', year('created_utc'))

In [9]:
submissions.show(5)

+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
| subreddit|score|        created_utc|hour_of_day|day_of_week|day_of_week_str|day_of_month|month|year|
+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
|television|    0|2021-01-27 22:16:47|         22|          4|      Wednesday|          27|    1|2021|
|     anime|    0|2021-01-27 22:17:30|         22|          4|      Wednesday|          27|    1|2021|
|television|    4|2021-01-27 22:17:42|         22|          4|      Wednesday|          27|    1|2021|
|    movies|    0|2021-01-27 22:18:23|         22|          4|      Wednesday|          27|    1|2021|
|     anime|    0|2021-01-27 22:19:23|         22|          4|      Wednesday|          27|    1|2021|
+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
only showing top 5 rows



In [11]:
# EDA on datetime features
# Group by subreddit, year, month and calculate average score
result_df = submissions.groupBy("subreddit", "year", "month").agg(avg("score").alias("average_score"))

In [12]:
# Show the result
result_df.show()

+----------+----+-----+------------------+
| subreddit|year|month|     average_score|
+----------+----+-----+------------------+
|    movies|2021|    5| 92.46727404587072|
|    movies|2021|   10|153.31222485607856|
|television|2021|    3| 326.3586521987436|
|television|2021|    6| 278.5865019011407|
|     anime|2021|    9| 71.18142039717267|
|television|2021|    7| 317.0394736842105|
|     anime|2021|   11| 73.64954381660661|
|    movies|2021|    6| 94.09434323148281|
|     anime|2021|    5| 76.54447121546112|
|    movies|2021|   11|137.37378399488745|
|     anime|2021|   10| 75.65115973454235|
|     anime|2021|    8| 69.77001127395715|
|    movies|2021|   12|119.85564542219346|
|television|2021|    4| 334.8437964891401|
|     anime|2021|    3|  75.2238120054958|
|    movies|2021|    4|103.05983768169105|
|    movies|2021|    8|163.04338364338363|
|television|2021|   11| 268.8965517241379|
|    movies|2021|    2|148.40551962604854|
|     anime|2021|    7|  71.8894794520548|
+----------

In [13]:
result_df_pd = result_df.toPandas()

In [15]:
result_df_pd.to_csv("../../data/csv/year_month_day_avgscore_eda.csv", index=False)

### Hour of day and Day of Week Data

In [16]:
submissions.show(5)

+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
| subreddit|score|        created_utc|hour_of_day|day_of_week|day_of_week_str|day_of_month|month|year|
+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
|television|    0|2021-01-27 22:16:47|         22|          4|      Wednesday|          27|    1|2021|
|     anime|    0|2021-01-27 22:17:30|         22|          4|      Wednesday|          27|    1|2021|
|television|    4|2021-01-27 22:17:42|         22|          4|      Wednesday|          27|    1|2021|
|    movies|    0|2021-01-27 22:18:23|         22|          4|      Wednesday|          27|    1|2021|
|     anime|    0|2021-01-27 22:19:23|         22|          4|      Wednesday|          27|    1|2021|
+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
only showing top 5 rows



In [17]:
# Group by subreddit, year, month and calculate average score
result_df1 = submissions.groupBy("subreddit", "hour_of_day", "day_of_week_str").agg(avg("score").alias("average_score"))

In [18]:
result_df1.show()

+----------+-----------+---------------+------------------+
| subreddit|hour_of_day|day_of_week_str|     average_score|
+----------+-----------+---------------+------------------+
|television|          0|      Wednesday|243.69258589511753|
|television|          3|         Friday| 174.2452380952381|
|     anime|         20|      Wednesday|22.687591240875914|
|     anime|          0|       Saturday| 46.98122065727699|
|     anime|          3|         Monday| 72.83072713057076|
|     anime|          6|         Monday|33.139367816091955|
|television|          5|         Monday|206.81736526946108|
|television|          9|         Friday| 253.1782945736434|
|television|         16|       Thursday|282.18665464382326|
|    movies|          9|         Friday| 83.85595238095237|
|    movies|          1|       Thursday|106.02503477051461|
|     anime|          3|       Thursday|  74.6719936076708|
|     anime|          9|        Tuesday|175.09882088714207|
|television|         21|       Saturday|

In [19]:
result_df1_pd = result_df1.toPandas()

In [20]:
result_df1_pd.to_csv("../../data/csv/daily_weekly_avgscore_eda.csv", index=False)

### Day of Month Data

In [21]:
submissions.show(5)

+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
| subreddit|score|        created_utc|hour_of_day|day_of_week|day_of_week_str|day_of_month|month|year|
+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
|television|    0|2021-01-27 22:16:47|         22|          4|      Wednesday|          27|    1|2021|
|     anime|    0|2021-01-27 22:17:30|         22|          4|      Wednesday|          27|    1|2021|
|television|    4|2021-01-27 22:17:42|         22|          4|      Wednesday|          27|    1|2021|
|    movies|    0|2021-01-27 22:18:23|         22|          4|      Wednesday|          27|    1|2021|
|     anime|    0|2021-01-27 22:19:23|         22|          4|      Wednesday|          27|    1|2021|
+----------+-----+-------------------+-----------+-----------+---------------+------------+-----+----+
only showing top 5 rows



In [22]:
# Group by subreddit, year, month and calculate average score
result_df2 = submissions.groupBy("subreddit", "day_of_month").count()

In [23]:
result_df2.show()

+----------+------------+-----+
| subreddit|day_of_month|count|
+----------+------------+-----+
|    movies|           4|12229|
|     anime|          27|12959|
|television|           9| 2893|
|    movies|          31| 7671|
|television|           1| 3017|
|    movies|          30|11524|
|     anime|          18|13014|
|     anime|          31| 7972|
|     anime|          17|12939|
|     anime|          10|13332|
|television|          23| 2971|
|television|          17| 2926|
|television|          13| 2942|
|    movies|           9|12423|
|television|          31| 1818|
|television|          15| 2857|
|television|          26| 2721|
|    movies|          10|12493|
|television|          29| 2586|
|    movies|          20|12414|
+----------+------------+-----+
only showing top 20 rows



In [27]:
result_df2_pd = result_df2.toPandas()

In [29]:
result_df2_pd.to_csv("../../data/csv/daily_weekly_count_eda.csv", index=False)